In [1]:
import warnings
warnings.simplefilter("ignore")
import pypsa
import time
import logging
import numpy as np
import os 
import sys
import yaml
import pyomo.environ as pyomo_env
import pandas as pd
from scipy.spatial import ConvexHull,  Delaunay
from scipy.interpolate import griddata
from multiprocessing import Lock, Process, Queue, current_process
import queue # imported for using queue.Empty exception
sys.path.append(os.getcwd())
import plotly.graph_objects as go
import plotly.figure_factory as ff
warnings.simplefilter("ignore")

In [58]:
network = pypsa.Network()
network.import_from_hdf5('PyPSA_project/data/networks/euro_00')
dim = 9

bus_list = ['DK','SE','NO','DE','PL','CZ','NL','AT','CH']

bus_list = bus_list[:dim]

for bus in network.buses.index:
    if bus not in bus_list:
        network.remove('Bus',name=bus)
        network.remove('Load',bus)

for link in network.links.index:
    if not (network.links.loc[link].bus0 in bus_list and network.links.loc[link].bus1 in bus_list ):
        network.remove('Link',link)

for generator in network.generators.index:
    if not network.generators.loc[generator].bus in bus_list:
        network.remove('Generator',generator)
        
for generator in network.generators.index:
    if network.generators.loc[generator].type == 'solar':
        #network.remove('Generator',generator)
        network.generators['type'][generator] = 'wind'

for generator in network.generators.index:
    if generator[-7:] == 'offwind':
        network.remove('Generator',generator)
snap_index = [5386, 7147,  782, 3211, 4402, 2189, 7307, 3426, 5932,  321, 4401,
       4223, 7726, 8511, 6275, 3611, 6101, 5524,   71, 8038, 6078,  227,
       1356, 4705]        
network.snapshots = network.snapshots[snap_index]

In [59]:
np.floor(np.random.rand(24)*8769).astype(int)

array([7997, 8694,  398, 1436, 7771, 4247,  862, 7503, 8754, 6719, 5463,
       8471, 7251, 7970, 5155, 1371, 1045, 5429, 7401, 5205,  190, 8417,
        557, 7660])

In [60]:
network.lopf(network.snapshots, 
            solver_name='gurobi'),

(('ok', 'optimal'),)

### Add CO2 constraint and solve again

In [61]:
co2_emission = [constraint.body() for constraint in network.model.global_constraints.values()][0]
co2_emission
network.remove('GlobalConstraint',"co2_limit")
target = (1-0.80)*co2_emission
network.add("GlobalConstraint","co2_limit",
      sense="<=",
      carrier_attribute="co2_emissions",
      constant=target)

In [62]:
network.lopf(network.snapshots, 
            solver_name='gurobi')
old_objective_value = network.model.objective()

In [63]:

var = []
for var_type in ['ocgt','wind']:#,'solar']:
    filter1 = network.generators.type==var_type
    var.append(sum(network.generators.p_nom_opt[filter1]))
initial_solution = var
initial_solution

[143224.84840872354, 420148.3564729163]

In [64]:
fig = go.Figure()


"""
fig.add_trace(go.Scattergeo(lon=[min(network.buses.x),max(network.buses.x)],
                            lat=[np.mean(network.buses.y),np.mean(network.buses.y)],
                            mode='lines'
                            ))
"""
# Links
import matplotlib.cm as cm
for link in network.links.iterrows():

    bus0 = network.buses.loc[link[1]['bus0']]
    bus1 = network.buses.loc[link[1]['bus1']]
    cap = link[1]['p_nom_opt']

    fig.add_trace(go.Scattergeo(
        locationmode = 'country names',
        lon = [bus0.x,bus1.x],
        lat = [bus0.y,bus1.y],
        mode = 'lines',
        line = dict(width = 2,color = 'green'),
        ))
    
    
# Nodes
fig.add_trace(go.Scattergeo(
    locationmode = 'country names',
    lon = network.buses.x,
    lat = network.buses.y,
    hoverinfo = 'text',
    text = network.buses.index,
    mode = 'text',
    marker = dict(
        size = 5,
        color = 'black',
        line = dict(
            width = 3,
            color = 'rgba(68, 68, 68, 0)'
        )),
     textfont={
        "color": "MidnightBlue",
        "family": "Balto, sans-serif",
        "size": 18
    },
    ))

"""
# Bar plots 
#network.generators.p_nom_opt=ds_co2_95.df_detail.iloc[0,0:111]


for bus in network.buses.iterrows():

    filter = [x and y for x,y in zip(network.generators.bus==bus[0],network.generators.type=='wind')]
    wind = sum(network.generators[filter].p_nom_opt)
    filter = [x and y for x,y in zip(network.generators.bus==bus[0],network.generators.type=='solar')]
    solar = sum(network.generators[filter].p_nom_opt)
    filter = [x and y for x,y in zip(network.generators.bus==bus[0],network.generators.type=='ocgt')]
    ocgt = sum(network.generators[filter].p_nom_opt)

    fig.add_trace(go.Scattergeo(
    locationmode = 'country names',
    lon = [bus[1]['x'],bus[1]['x']+0.5,bus[1]['x']-0.5 ],
    lat = [bus[1]['y'], bus[1]['y'],bus[1]['y']],
    hoverinfo = 'text',
    text = bus[0],
    mode = 'markers',
    marker = dict(
        size = np.array([wind,solar,ocgt])/2000,
        color = ['blue','yellow','black'],
        symbol = 'line-ns',
        line = dict(
            width = 10,
            color = ['blue','yellow','black'],
        )
    )))
"""
"""
# Legend 
fig.add_trace(go.Scattergeo(
locationmode = 'country names',
lon = [-11,-11,-11],
lat = [62,63,64],
hoverinfo = 'text',
text = 'legend',
mode = 'markers',
marker = dict(
    size = 10,
    color = ['blue','yellow','black'],
    symbol = 'line-ns',
    line = dict(
        width = 10,
        color = ['blue','yellow','black'],
    )
)))
"""


fig.update_layout(
    showlegend = False,
    geo = go.layout.Geo(
        scope = 'europe',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
        lataxis = dict(
            range = [45, 62],
            showgrid = False
        ),
        lonaxis = dict(
            range = [0, 24],
            showgrid = False
        )
    ),
)
fig.update_layout(
    autosize=False,
    showlegend=False,
    width=500,
    height=500,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=0.5,y=1)
)

fig.show()
fig.write_image("C:/Users\Tim\OneDrive - Aarhus universitet\Speciale\Report\Images\comparison_topology.pdf")

In [87]:
def plot_network(solutions):
    from plotly.subplots import make_subplots
    fig = make_subplots(
        rows=3, cols=2,
        #column_widths=[0.6, 0.4],
        row_heights=[0.45,0.45,0.15],
        subplot_titles=['','','(a)','(b)','(c)','(d)'],
        specs=[[{"type": 'Scattergeo'},{"type": 'Scattergeo'}],
               [ {"type": 'Scattergeo'},{"type": 'Scattergeo'}],
              [ {"type": 'Scatter'},{"type": 'Scatter'}]])


    """
    fig.add_trace(go.Scattergeo(lon=[min(network.buses.x),max(network.buses.x)],
                                lat=[np.mean(network.buses.y),np.mean(network.buses.y)],
                                mode='lines'
                                ))
    """

    for i in range(4):
        try:
            network = solutions[i]
        except:
            break
        # Links
        import matplotlib.cm as cm
        for link in network.links.iterrows():

            bus0 = network.buses.loc[link[1]['bus0']]
            bus1 = network.buses.loc[link[1]['bus1']]
            cap = link[1]['p_nom_opt']
            cap_max = max(network.links.p_nom_opt)

            fig.add_trace(go.Scattergeo(
                locationmode = 'country names',
                geo = 'geo'+str(i+1),
                lon = [bus0.x,bus1.x],
                lat = [bus0.y,bus1.y],
                mode = 'lines',
                line = dict(width = cap/cap_max*5+0.5,color = 'green'),
                ),row=int(np.floor(i/2)+1),col=i%2+1)


        # Nodes
        """
        fig.add_trace(go.Scattergeo(
            locationmode = 'country names',
            lon = network.buses.x,
            lat = network.buses.y,
            hoverinfo = 'text',
            text = network.buses.index,
            mode = 'markers',
            marker = dict(
                size = 5,
                color = 'black',
                line = dict(
                    width = 3,
                    color = 'rgba(68, 68, 68, 0)'
                )),
             textfont={
                "color": "MidnightBlue",
                "family": "Balto, sans-serif",
                "size": 18
            },
            ))
        """

        # Bar plots 
        #network.generators.p_nom_opt=ds_co2_95.df_detail.iloc[0,0:111]


        for bus in network.buses.iterrows():

            filter = [x and y for x,y in zip(network.generators.bus==bus[0],network.generators.type=='wind')]
            wind = sum(network.generators[filter].p_nom_opt)
            filter = [x and y for x,y in zip(network.generators.bus==bus[0],network.generators.type=='solar')]
            solar = sum(network.generators[filter].p_nom_opt)
            filter = [x and y for x,y in zip(network.generators.bus==bus[0],network.generators.type=='ocgt')]
            ocgt = sum(network.generators[filter].p_nom_opt)

            fig.add_trace(go.Scattergeo(
            locationmode = 'country names',
            lon = [bus[1]['x']-0.25,bus[1]['x']+0.25,bus[1]['x']-0.5 ],
            lat = [bus[1]['y'], bus[1]['y'],bus[1]['y']],
            hoverinfo = 'text',
            geo = 'geo'+str(i+1),
            text = bus[0],
            mode = 'markers',
            marker = dict(
                size = np.array([wind,ocgt])/4000,
                color = ['blue','yellow','black'],
                symbol = 'line-ns',
                line = dict(
                    width = 10,
                    color = ['orange','black'],
                )
            )),row=int(np.floor(i/2)+1),col=i%2+1)

    """
    # Legend 
    fig.add_trace(go.Scattergeo(
    locationmode = 'country names',
    lon = [-11,-11,-11],
    lat = [62,63,64],
    hoverinfo = 'text',
    text = 'legend',
    mode = 'markers',
    marker = dict(
        size = 10,
        color = ['blue','yellow','black'],
        symbol = 'line-ns',
        line = dict(
            width = 10,
            color = ['blue','yellow','black'],
        )
    )))
    """
    #Legend
    # Capacity
    fig.add_trace(go.Scatter(
        x = [0.1,0.13,0.16,0.3,0.33,0.36],
        y = [6,6,6,6,6,6],
        mode = 'markers',
        marker = dict(
            size = [25,12.5,5,25,12.5,5],
            symbol = 'line-ns',
            line = dict(
                width = 7,
                color = ['black','black','black','orange','orange','orange',],
            )),
    ),row=3,col=1)

    # Transmission
    fig.add_trace(go.Scatter(
        x = [0.5,0.53,0.56,],
        y = [6,6,6],
        mode = 'markers',
        marker = dict(
            size = 15,
            symbol = 'line-ew',
            line = dict(
                width = [25000/cap_max*5+0.5,10000/cap_max*5+0.5,0.5],
                color = ['green','green','green'],
            )),
    ),row=3,col=1)
    # Text
    fig.add_trace(go.Scatter(
        x = [0.13,0.33,0.52,0.1,0.13,0.16,0.3,0.33,0.36,0.5,0.53,0.56,],
        y = [9,9,9,3,3,3,3,3,3,3,3,3],
        text = ['OCGT [GW]','VRES [GW]','Transmission [GW]','100','50','20','100','50','20','25','10','0'],
        mode = 'text',
        
    ),row=3,col=1)

    fig.update_geos(
            scope = 'europe',
            projection_type = 'azimuthal equal area',
            showland = True,
            landcolor = 'rgb(243, 243, 243)',
            countrycolor = 'rgb(204, 204, 204)',
            showocean=False,
            #domain=dict(x=[0,1],y=[0,1]),
            lataxis = dict(
                range = [45, 62],
                showgrid = False
            ),
            lonaxis = dict(
                range = [0, 24],
                showgrid = False
            ))
    fig.update_layout(
        geo2=dict(
            domain=dict(x=[0.52,0.99],y=[0.55,1])),#Top Rigth
        geo1=dict(
            domain=dict(x=[0,0.48],y=[0.55,1])),#Top Left
        geo3=dict(
            domain=dict(x=[0,0.48],y=[0.1,0.55])),#Botom left
        geo4=dict(
            domain=dict(x=[0.52,0.99],y=[0.1,0.55])),#Botom right
    )

    fig.update_layout(
        autosize=False,
        showlegend=False,
        xaxis=dict(showticklabels=False,range=[0,0.7],domain=[0,1]),
        yaxis=dict(showticklabels=False,range=[0,10],domain=[0,0.08]),
            paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(0,0,0,0)',
        width=1000,
        height=1000,
        margin=dict(l=5, r=5, t=5, b=5,pad=0),
        )

    return fig
    

In [78]:
def direction_search(network, snapshots,options,direction): #  MGA_slack = 0.05, point=[0,0,0],dim=3,old_objective_value=0):
# Identify the nonzero decision variables that should enter the MGA objective function.
    old_objective_value = options['old_objective_value']
    dim = options['dim']
    MGA_slack = 0.1

    if dim == 3:
        generators = [gen_p for gen_p in network.model.generator_p_nom]
        types = ['ocgt','wind','olar']
        variables = []
        for i in range(3):
            gen_p_type = [gen_p  for gen_p in generators if gen_p[-4:]==types[i]]
            variables.append(sum([network.model.generator_p_nom[gen_p] for gen_p in gen_p_type]))
            
                             
    elif dim == 2:
        generators = [gen_p for gen_p in network.model.generator_p_nom]
        variables = []

        gen_p_type = [gen_p  for gen_p in generators if gen_p[-4:]=='ocgt']
        variables.append(sum([network.model.generator_p_nom[gen_p] for gen_p in gen_p_type]))
        
        gen_p_type = [gen_p  for gen_p in generators if gen_p[-4:]=='wind' or gen_p[-4:]=='olar']
        variables.append(sum([network.model.generator_p_nom[gen_p] for gen_p in gen_p_type]))
    else :
        variables = [network.model.generator_p_nom[gen_p] for gen_p in network.model.generator_p_nom]
        
    objective = 0
    for i in range(len(variables)):
        #print(variables[i])
        objective += direction[i]*variables[i]

    print(objective)
    # Add the new MGA objective function to the model.
    #objective += network.model.objective.expr * 1e-9
    network.model.mga_objective = pyomo_env.Objective(expr=objective)
    # Deactivate the old objective function and activate the MGA objective function.
    network.model.objective.deactivate()
    network.model.mga_objective.activate()
    # Add the MGA slack constraint.
    network.model.mga_constraint = pyomo_env.Constraint(expr=network.model.objective.expr <= 
                                          (1 + MGA_slack) * old_objective_value)


In [67]:
logging.disable()

In [79]:
#logging.disable()
data_detail_sum = []
options = dict(dim=2,old_objective_value=old_objective_value)
dim=2
old_volume = 0
epsilon = 1
solutions_sum = []
while epsilon > 0.001:
    data_detail = list(data_detail_sum)
    if len(data_detail_sum)<=1 : # if only original solution exists, max/min directions are chosen
        directions = np.concatenate([np.diag(np.ones(dim)),-np.diag(np.ones(dim))],axis=0)
    else : # Otherwise search in directions normal to faces
        directions = np.array(hull.equations)[:,0:-1]
    # Itterate over directions in batch 
    for direction in directions:
        #print(direction)
        direction = np.concatenate([direction,[0]])
        network.lopf(network.snapshots,                                 
                        solver_name='gurobi',                                 
                        extra_functionality=lambda network,                                 
                        snapshots: direction_search(network,snapshots,options,direction))
        solutions_sum.append(network.copy())
        var = []
        for var_type in ['ocgt','wind']:#,'solar']:
            filter1 = network.generators.type==var_type
            var.append(sum(network.generators.p_nom_opt[filter1]))
        data_detail_sum.append(var)
    

    hull = ConvexHull(data_detail_sum)#,qhull_options='QJ')
    
    delta_v = hull.volume - old_volume
    old_volume = hull.volume
    epsilon = delta_v/hull.volume
    
    print(epsilon)


data_detail_sum = np.array(data_detail_sum)

generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + generator_p_nom[SE ocgt]
generator_p_nom[AT onwind] + generator_p_nom[AT solar] + generator_p_nom[CH onwind] + generator_p_nom[CH solar] + generator_p_nom[CZ onwind] + generator_p_nom[CZ solar] + generator_p_nom[DE onwind] + generator_p_nom[DE solar] + generator_p_nom[DK onwind] + generator_p_nom[DK solar] + generator_p_nom[NL onwind] + generator_p_nom[NL solar] + generator_p_nom[NO onwind] + generator_p_nom[NO solar] + generator_p_nom[PL onwind] + generator_p_nom[PL solar] + generator_p_nom[SE onwind] + generator_p_nom[SE solar]
- (generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + generator_p_nom[SE ocgt])
- (

-0.292570889673848*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + generator_p_nom[SE ocgt]) - 0.956243836328085*(generator_p_nom[AT onwind] + generator_p_nom[AT solar] + generator_p_nom[CH onwind] + generator_p_nom[CH solar] + generator_p_nom[CZ onwind] + generator_p_nom[CZ solar] + generator_p_nom[DE onwind] + generator_p_nom[DE solar] + generator_p_nom[DK onwind] + generator_p_nom[DK solar] + generator_p_nom[NL onwind] + generator_p_nom[NL solar] + generator_p_nom[NO onwind] + generator_p_nom[NO solar] + generator_p_nom[PL onwind] + generator_p_nom[PL solar] + generator_p_nom[SE onwind] + generator_p_nom[SE solar])
0.9636968046746268*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt

0.5863262846870927*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + generator_p_nom[SE ocgt]) + 0.8100749890504153*(generator_p_nom[AT onwind] + generator_p_nom[AT solar] + generator_p_nom[CH onwind] + generator_p_nom[CH solar] + generator_p_nom[CZ onwind] + generator_p_nom[CZ solar] + generator_p_nom[DE onwind] + generator_p_nom[DE solar] + generator_p_nom[DK onwind] + generator_p_nom[DK solar] + generator_p_nom[NL onwind] + generator_p_nom[NL solar] + generator_p_nom[NO onwind] + generator_p_nom[NO solar] + generator_p_nom[PL onwind] + generator_p_nom[PL solar] + generator_p_nom[SE onwind] + generator_p_nom[SE solar])
-0.5852108437514766*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO oc

-0.9944419549329425*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + generator_p_nom[SE ocgt]) - 0.10528626818891271*(generator_p_nom[AT onwind] + generator_p_nom[AT solar] + generator_p_nom[CH onwind] + generator_p_nom[CH solar] + generator_p_nom[CZ onwind] + generator_p_nom[CZ solar] + generator_p_nom[DE onwind] + generator_p_nom[DE solar] + generator_p_nom[DK onwind] + generator_p_nom[DK solar] + generator_p_nom[NL onwind] + generator_p_nom[NL solar] + generator_p_nom[NO onwind] + generator_p_nom[NO solar] + generator_p_nom[PL onwind] + generator_p_nom[PL solar] + generator_p_nom[SE onwind] + generator_p_nom[SE solar])
-0.9248894107070599*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO 

0.04500974636081048*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + generator_p_nom[SE ocgt]) + 0.9989865478236108*(generator_p_nom[AT onwind] + generator_p_nom[AT solar] + generator_p_nom[CH onwind] + generator_p_nom[CH solar] + generator_p_nom[CZ onwind] + generator_p_nom[CZ solar] + generator_p_nom[DE onwind] + generator_p_nom[DE solar] + generator_p_nom[DK onwind] + generator_p_nom[DK solar] + generator_p_nom[NL onwind] + generator_p_nom[NL solar] + generator_p_nom[NO onwind] + generator_p_nom[NO solar] + generator_p_nom[PL onwind] + generator_p_nom[PL solar] + generator_p_nom[SE onwind] + generator_p_nom[SE solar])
0.1292089516535194*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO oc

0.0013109716941292439
-5.424680629449985e-16*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + generator_p_nom[SE ocgt]) - (generator_p_nom[AT onwind] + generator_p_nom[AT solar] + generator_p_nom[CH onwind] + generator_p_nom[CH solar] + generator_p_nom[CZ onwind] + generator_p_nom[CZ solar] + generator_p_nom[DE onwind] + generator_p_nom[DE solar] + generator_p_nom[DK onwind] + generator_p_nom[DK solar] + generator_p_nom[NL onwind] + generator_p_nom[NL solar] + generator_p_nom[NO onwind] + generator_p_nom[NO solar] + generator_p_nom[PL onwind] + generator_p_nom[PL solar] + generator_p_nom[SE onwind] + generator_p_nom[SE solar])
-0.11850616949707116*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_n

0.7694076780737629*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + generator_p_nom[SE ocgt]) + 0.6387580331558588*(generator_p_nom[AT onwind] + generator_p_nom[AT solar] + generator_p_nom[CH onwind] + generator_p_nom[CH solar] + generator_p_nom[CZ onwind] + generator_p_nom[CZ solar] + generator_p_nom[DE onwind] + generator_p_nom[DE solar] + generator_p_nom[DK onwind] + generator_p_nom[DK solar] + generator_p_nom[NL onwind] + generator_p_nom[NL solar] + generator_p_nom[NO onwind] + generator_p_nom[NO solar] + generator_p_nom[PL onwind] + generator_p_nom[PL solar] + generator_p_nom[SE onwind] + generator_p_nom[SE solar])
0.045010087927943594*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO o

-0.9629817854517777*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + generator_p_nom[SE ocgt]) + 0.2695664684045599*(generator_p_nom[AT onwind] + generator_p_nom[AT solar] + generator_p_nom[CH onwind] + generator_p_nom[CH solar] + generator_p_nom[CZ onwind] + generator_p_nom[CZ solar] + generator_p_nom[DE onwind] + generator_p_nom[DE solar] + generator_p_nom[DK onwind] + generator_p_nom[DK solar] + generator_p_nom[NL onwind] + generator_p_nom[NL solar] + generator_p_nom[NO onwind] + generator_p_nom[NO solar] + generator_p_nom[PL onwind] + generator_p_nom[PL solar] + generator_p_nom[SE onwind] + generator_p_nom[SE solar])
-0.9925670849272483*(generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO o

In [81]:
fig = go.Figure()



fig.add_trace(go.Scatter(x=hull.points[np.append(hull.vertices,hull.vertices[0])][:,0]/1000,
                          y=hull.points[np.append(hull.vertices,hull.vertices[0])][:,1]/1000,
                          #z=data_detail_sum[:,2],
                          mode='lines',name='Convex hull',fill='tonexty',fillcolor='rgba(255,0,0,0.3)'))

fig.add_trace(go.Scatter(x=data_detail_sum[:,0]/1000,
                          y=data_detail_sum[:,1]/1000,
                          #z=data_detail_sum[:,2],
                          mode='markers',name='MGA solutions'))

fig.add_trace(go.Scatter(x=[initial_solution[0]/1000],
                          y=[initial_solution[1]/1000],
                          #z=[np.mean(data_detail_sum,axis=0)[2]],
                          mode='markers',name='Initial_solution'))

fig.update_yaxes(title_text='vres capacity installed [GW]',range=[350,550])
fig.update_xaxes(title_text='ocgt capacity installed [GW]',range=[80,300])
fig.update_layout(
    autosize=False,
    showlegend=True,
    width=500,
    height=500,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=0.6,y=1)
)
fig.write_image("C:/Users\Tim\OneDrive - Aarhus universitet\Speciale\Report\Images\comparison_1.pdf")
fig.show()

In [82]:
fig = plot_network(solutions_sum)
fig.write_image("C:/Users\Tim\OneDrive - Aarhus universitet\Speciale\Report\Images\MGA_sum.pdf")
fig.show()

### HSJ

In [83]:
import pyomo.environ as pyomo_env
MGA_slack = 0.1
solutions_hsj = []
# Defining exstra functionality, that updates the objective function of the network
def extra_functionality(network, snapshots,  MGA_slack = 0.05):
    # Identify the nonzero decision variables that should enter the MGA objective function.
    generator_outputs = network.generators.p_nom_opt
    nonzero_gen_p = list()
    for gen_p in network.model.generator_p_nom :
        if generator_outputs[gen_p] > 0 :
            nonzero_gen_p.append(gen_p)
    # Build new MGA objective function.
    MGA_objective = 0
    for gen_p in nonzero_gen_p:
        MGA_objective += network.model.generator_p_nom[gen_p]
    # Add the new MGA objective function to the model.
    network.model.mga_objective = pyomo_env.Objective(expr=MGA_objective)
    # Deactivate the old objective function and activate the MGA objective function.
    network.model.objective.deactivate()
    network.model.mga_objective.activate()
    # Add the MGA slack constraint.
    #print('old objective value ',old_objective_value)
    network.model.mga_constraint = pyomo_env.Constraint(expr=network.model.objective.expr <= 
                                          (1 + MGA_slack) * old_objective_value)


# Initial solution of network, with non MGA objective function 
network.lopf(network.snapshots,solver_name='gurobi')
# Saving the value of the old objective function.
old_objective_value = network.model.objective()
print('old objective valeu: ' + str(old_objective_value))
# Define a list of soltutions to the network object containing coppys of the network
network.solutions = []
data_detail_hsj = []
# Loop until a non original solution is found
original_solution = True
i = 0
while original_solution:
    # Save current version of network in solutions
    network.solutions.append(network.copy())
    solutions_hsj.append(network.copy())
    #network.solutions[i].objective_value = network.model.objective()
    #Solve network with updated objective function
    network.lopf(network.snapshots,\
                 solver_name='gurobi',\
                 extra_functionality=lambda network,\
                 snapshots: extra_functionality(network, snapshots, MGA_slack)) 
    # Tjek if the solution is seen before
    for i in range(len(network.solutions)):
        if network.generators_t.p.equals(network.solutions[i].generators_t.p):
            original_solution = False
            
    var = []
    for var_type in ['ocgt','wind']:#,'solar']:
        filter1 = network.generators.type==var_type
        var.append(sum(network.generators.p_nom_opt[filter1]))
    data_detail_hsj.append(var)

    i += 1
data_detail_hsj = np.array(data_detail_hsj)

old objective valeu: 68411982466.69762


In [86]:
solutions_hsj

[Network , Network , Network ]

In [84]:
fig = go.Figure()



fig.add_trace(go.Scatter(x=hull.points[np.append(hull.vertices,hull.vertices[0])][:,0]/1000,
                          y=hull.points[np.append(hull.vertices,hull.vertices[0])][:,1]/1000,
                          #z=data_detail_sum[:,2],
                          mode='lines',name='Convex hull'))

fig.add_trace(go.Scatter(x=data_detail_hsj[:,0]/1000,
                          y=data_detail_hsj[:,1]/1000,
                          #z=data_detail_sum[:,2],
                          mode='markers',name='MGA solutions'))

fig.add_trace(go.Scatter(x=[initial_solution[0]/1000],
                          y=[initial_solution[1]/1000],
                          #z=[np.mean(data_detail_sum,axis=0)[2]],
                          mode='markers',name='Initial_solution'))

fig.update_yaxes(title_text='vres capacity installed [GW]',range=[350,550])
fig.update_xaxes(title_text='ocgt capacity installed [GW]',range=[80,300])
fig.update_layout(
    autosize=False,
    showlegend=True,
    width=500,
    height=500,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=0.6,y=1)
)
fig.write_image("C:/Users\Tim\OneDrive - Aarhus universitet\Speciale\Report\Images\comparison_2.pdf")
fig.show()

In [88]:
fig = plot_network(solutions_hsj)
fig.show()
fig.write_image("C:/Users\Tim\OneDrive - Aarhus universitet\Speciale\Report\Images\HSJ.pdf")

### Max Min of all variables

In [89]:
#logging.disable()
data_detail_sum = []
options = dict(dim=24,old_objective_value=old_objective_value)
dim=27
old_volume = 0
epsilon = 1
solutions_maxmin = []
while epsilon > 0.001:
    data_detail = list(data_detail_sum)
    if len(data_detail_sum)<=1 : # if only original solution exists, max/min directions are chosen
        directions = np.concatenate([np.diag(np.ones(dim)),-np.diag(np.ones(dim))],axis=0)
    else : # Otherwise search in directions normal to faces
        directions = np.array(hull.equations)[:,0:-1]
    # Itterate over directions in batch 
    for direction in directions:
        print(direction)
        direction = np.concatenate([direction,[0]])
        network.lopf(network.snapshots,                                 
                        solver_name='gurobi',                                 
                        extra_functionality=lambda network,                                 
                        snapshots: direction_search(network,snapshots,options,direction))
        solutions_maxmin.append(network.copy())
        var = []
        for var_type in ['ocgt','wind']:#,'solar']:
            filter1 = network.generators.type==var_type
            var.append(sum(network.generators.p_nom_opt[filter1]))
        data_detail_sum.append(var)
    
    break
    hull = ConvexHull(data_detail_sum)#,qhull_options='QJ')
    
    delta_v = hull.volume - old_volume
    old_volume = hull.volume
    epsilon = delta_v/hull.volume
    
    print(epsilon)


data_detail_sum = np.array(data_detail_sum)

[1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
generator_p_nom[AT onwind]
[0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
generator_p_nom[AT solar]
[0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
generator_p_nom[AT ocgt]
[0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
generator_p_nom[CH onwind]
[0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
generator_p_nom[CH solar]
[0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
generator_p_nom[CH ocgt]
[0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
generator_p_nom[CZ onwind]
[0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
generator_p_nom[CZ solar]
[0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0.]
generator_p_nom[CZ ocgt]
[0. 0. 0. 

In [91]:
fig = go.Figure()



fig.add_trace(go.Scatter(x=hull.points[np.append(hull.vertices,hull.vertices[0])][:,0]/1000,
                          y=hull.points[np.append(hull.vertices,hull.vertices[0])][:,1]/1000,
                          #z=data_detail_sum[:,2],
                          mode='lines',name='Convex hull'))

fig.add_trace(go.Scatter(x=data_detail_sum[:,0]/1000,
                          y=data_detail_sum[:,1]/1000,
                          #z=data_detail_sum[:,2],
                          mode='markers',name='MGA solutions'))

fig.add_trace(go.Scatter(x=[initial_solution[0]/1000],
                          y=[initial_solution[1]/1000],
                          #z=[np.mean(data_detail_sum,axis=0)[2]],
                          mode='markers',name='Initial_solution'))

fig.update_yaxes(title_text='vres capacity installed [GW]',range=[350,550])
fig.update_xaxes(title_text='ocgt capacity installed [GW]',range=[80,300])
fig.update_layout(
    autosize=False,
    showlegend=True,
    width=500,
    height=500,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=0.6,y=1)
)
fig.write_image("C:/Users\Tim\OneDrive - Aarhus universitet\Speciale\Report\Images\comparison_3.pdf")
fig.show()

### Max Min of grouped variables

In [92]:
#logging.disable()
data_detail_sum = []
options = dict(dim=2,old_objective_value=old_objective_value)
dim=2
old_volume = 0
epsilon = 1
while epsilon > 0.001:
    data_detail = list(data_detail_sum)
    if len(data_detail_sum)<=1 : # if only original solution exists, max/min directions are chosen
        directions = np.concatenate([np.diag(np.ones(dim)),-np.diag(np.ones(dim))],axis=0)
    else : # Otherwise search in directions normal to faces
        directions = np.array(hull.equations)[:,0:-1]
    # Itterate over directions in batch 
    for direction in directions:
        print(direction)
        direction = np.concatenate([direction,[0]])
        network.lopf(network.snapshots,                                 
                        solver_name='gurobi',                                 
                        extra_functionality=lambda network,                                 
                        snapshots: direction_search(network,snapshots,options,direction))

        var = []
        for var_type in ['ocgt','wind']:#,'solar']:
            filter1 = network.generators.type==var_type
            var.append(sum(network.generators.p_nom_opt[filter1]))
        data_detail_sum.append(var)
    
    break
    hull = ConvexHull(data_detail_sum)#,qhull_options='QJ')
    
    delta_v = hull.volume - old_volume
    old_volume = hull.volume
    epsilon = delta_v/hull.volume
    
    print(epsilon)


data_detail_sum = np.array(data_detail_sum)

[1. 0.]
generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + generator_p_nom[SE ocgt]
[0. 1.]
generator_p_nom[AT onwind] + generator_p_nom[AT solar] + generator_p_nom[CH onwind] + generator_p_nom[CH solar] + generator_p_nom[CZ onwind] + generator_p_nom[CZ solar] + generator_p_nom[DE onwind] + generator_p_nom[DE solar] + generator_p_nom[DK onwind] + generator_p_nom[DK solar] + generator_p_nom[NL onwind] + generator_p_nom[NL solar] + generator_p_nom[NO onwind] + generator_p_nom[NO solar] + generator_p_nom[PL onwind] + generator_p_nom[PL solar] + generator_p_nom[SE onwind] + generator_p_nom[SE solar]
[-1. -0.]
- (generator_p_nom[AT ocgt] + generator_p_nom[CH ocgt] + generator_p_nom[CZ ocgt] + generator_p_nom[DE ocgt] + generator_p_nom[DK ocgt] + generator_p_nom[NL ocgt] + generator_p_nom[NO ocgt] + generator_p_nom[PL ocgt] + gen

In [93]:
fig = go.Figure()



fig.add_trace(go.Scatter(x=hull.points[np.append(hull.vertices,hull.vertices[0])][:,0]/1000,
                          y=hull.points[np.append(hull.vertices,hull.vertices[0])][:,1]/1000,
                          #z=data_detail_sum[:,2],
                          mode='lines',name='Convex hull'))

fig.add_trace(go.Scatter(x=data_detail_sum[:,0]/1000,
                          y=data_detail_sum[:,1]/1000,
                          #z=data_detail_sum[:,2],
                          mode='markers',name='MGA solutions'))

fig.add_trace(go.Scatter(x=[initial_solution[0]/1000],
                          y=[initial_solution[1]/1000],
                          #z=[np.mean(data_detail_sum,axis=0)[2]],
                          mode='markers',name='Initial_solution'))

fig.update_yaxes(title_text='vres capacity installed [GW]',range=[350,550])
fig.update_xaxes(title_text='ocgt capacity installed [GW]',range=[80,300])
fig.update_layout(
    autosize=False,
    showlegend=True,
    width=500,
    height=500,
    paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    legend=dict(x=0.6,y=1)
)
fig.write_image("C:/Users\Tim\OneDrive - Aarhus universitet\Speciale\Report\Images\comparison_4.pdf")
fig.show()

In [ ]:
from plotly.subplots import make_subplots
fig = make_subplots(
    rows=3, cols=2,
    #column_widths=[0.6, 0.4],
    row_heights=[0.45,0.45,0.15],
    subplot_titles=['','','(a)','(b)','(c)','(d)'],
    specs=[[{"type": 'Scattergeo'},{"type": 'Scattergeo'}],
           [ {"type": 'Scattergeo'},{"type": 'Scattergeo'}],
          [ {"type": 'Scatter'},{"type": 'Scatter'}]])


"""
fig.add_trace(go.Scattergeo(lon=[min(network.buses.x),max(network.buses.x)],
                            lat=[np.mean(network.buses.y),np.mean(network.buses.y)],
                            mode='lines'
                            ))
"""

for i in range(4):
    network = solutions[i]
    # Links
    import matplotlib.cm as cm
    for link in network.links.iterrows():

        bus0 = network.buses.loc[link[1]['bus0']]
        bus1 = network.buses.loc[link[1]['bus1']]
        cap = link[1]['p_nom_opt']
        cap_max = max(network.links.p_nom_opt)

        fig.add_trace(go.Scattergeo(
            locationmode = 'country names',
            geo = 'geo'+str(i+1),
            lon = [bus0.x,bus1.x],
            lat = [bus0.y,bus1.y],
            mode = 'lines',
            line = dict(width = cap/cap_max*5+0.5,color = 'green'),
            ),row=int(np.floor(i/2)+1),col=i%2+1)


    # Nodes
    """
    fig.add_trace(go.Scattergeo(
        locationmode = 'country names',
        lon = network.buses.x,
        lat = network.buses.y,
        hoverinfo = 'text',
        text = network.buses.index,
        mode = 'markers',
        marker = dict(
            size = 5,
            color = 'black',
            line = dict(
                width = 3,
                color = 'rgba(68, 68, 68, 0)'
            )),
         textfont={
            "color": "MidnightBlue",
            "family": "Balto, sans-serif",
            "size": 18
        },
        ))
    """

    # Bar plots 
    #network.generators.p_nom_opt=ds_co2_95.df_detail.iloc[0,0:111]


    for bus in network.buses.iterrows():

        filter = [x and y for x,y in zip(network.generators.bus==bus[0],network.generators.type=='wind')]
        wind = sum(network.generators[filter].p_nom_opt)
        filter = [x and y for x,y in zip(network.generators.bus==bus[0],network.generators.type=='solar')]
        solar = sum(network.generators[filter].p_nom_opt)
        filter = [x and y for x,y in zip(network.generators.bus==bus[0],network.generators.type=='ocgt')]
        ocgt = sum(network.generators[filter].p_nom_opt)

        fig.add_trace(go.Scattergeo(
        locationmode = 'country names',
        lon = [bus[1]['x'],bus[1]['x']+0.5,bus[1]['x']-0.5 ],
        lat = [bus[1]['y'], bus[1]['y'],bus[1]['y']],
        hoverinfo = 'text',
        geo = 'geo'+str(i+1),
        text = bus[0],
        mode = 'markers',
        marker = dict(
            size = np.array([wind,ocgt])/2000,
            color = ['blue','yellow','black'],
            symbol = 'line-ns',
            line = dict(
                width = 10,
                color = ['orange','black'],
            )
        )),row=int(np.floor(i/2)+1),col=i%2+1)

"""
# Legend 
fig.add_trace(go.Scattergeo(
locationmode = 'country names',
lon = [-11,-11,-11],
lat = [62,63,64],
hoverinfo = 'text',
text = 'legend',
mode = 'markers',
marker = dict(
    size = 10,
    color = ['blue','yellow','black'],
    symbol = 'line-ns',
    line = dict(
        width = 10,
        color = ['blue','yellow','black'],
    )
)))
"""
#Legend
# Capacity
fig.add_trace(go.Scatter(
    x = [0.1,0.13,0.16,0.3,0.33,0.36],
    y = [6,6,6,6,6,6],
    mode = 'markers',
    marker = dict(
        size = [25,12.5,5,25,12.5,5],
        symbol = 'line-ns',
        line = dict(
            width = 7,
            color = ['black','black','black','orange','orange','orange',],
        )),
),row=3,col=1)

# Transmission
fig.add_trace(go.Scatter(
    x = [0.5,0.53,0.56,],
    y = [6,6,6],
    mode = 'markers',
    marker = dict(
        size = 15,
        symbol = 'line-ew',
        line = dict(
            width = [25000/cap_max*5+0.5,10000/cap_max*5+0.5,0.5],
            color = ['green','green','green'],
        )),
),row=3,col=1)
# Text
fig.add_trace(go.Scatter(
    x = [0.13,0.33,0.52,0.1,0.13,0.16,0.3,0.33,0.36,0.5,0.53,0.56,],
    y = [9,9,9,3,3,3,3,3,3,3,3,3],
    text = ['VRES [GW]','OCGT [GW]','Transmission [GW]','50','25','10','50','25','10','25','10','0'],
    mode = 'text',
    textposition="middle center"
),row=3,col=1)

fig.update_geos(
        scope = 'europe',
        projection_type = 'azimuthal equal area',
        showland = True,
        landcolor = 'rgb(243, 243, 243)',
        countrycolor = 'rgb(204, 204, 204)',
        showocean=False,
        #domain=dict(x=[0,1],y=[0,1]),
        lataxis = dict(
            range = [45, 62],
            showgrid = False
        ),
        lonaxis = dict(
            range = [0, 24],
            showgrid = False
        ))
fig.update_layout(
    geo1=dict(
        domain=dict(x=[0.52,0.99],y=[0.55,1])),
    geo2=dict(
        domain=dict(x=[0,0.48],y=[0.55,1])),
    geo3=dict(
        domain=dict(x=[0,0.48],y=[0.1,0.55])),
    geo4=dict(
        domain=dict(x=[0.52,0.99],y=[0.1,0.55])),
)

fig.update_layout(
    autosize=False,
    showlegend=False,
    xaxis=dict(showticklabels=False,range=[0,0.7],domain=[0,1]),
    yaxis=dict(showticklabels=False,range=[0,10],domain=[0,0.08]),
        paper_bgcolor='rgba(0,0,0,0)',
    plot_bgcolor='rgba(0,0,0,0)',
    width=1000,
    height=1000,
    margin=dict(l=5, r=5, t=5, b=5,pad=0),
    )
fig.show()

